In [1]:
!pip install bs4
!pip install lxml
!pip install html5lib

  Running setup.py bdist_wheel for bs4: started
  Running setup.py bdist_wheel for bs4: finished with status 'done'
  Stored in directory: C:\Users\Acer\AppData\Local\pip\Cache\wheels\a0\b0\b2\4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import numpy as np

In [8]:
dates=pd.date_range(start='1958-08-24', end='2010-10-01', freq='2W')
a=list(dates.date)

In [9]:
urls=[]
base="https://www.billboard.com/charts/hot-100/"
for i in a:
    url=base+i.strftime("%Y-%m-%d")
    urls.append(url)

In [11]:
def get_songs(soup):
    x=soup.find_all('span', class_='chart-element__information__song text--truncate color--primary')
    songs=[]
    for i in x:
        i=i.text.strip()
        songs.append(i)
    return songs

In [12]:
def get_artists(soup):
    x=soup.find_all('span', class_='chart-element__information__artist text--truncate color--secondary')
    artists=[]
    for i in x:
        i=i.text.strip()
        artists.append(i)
    return artists

In [ ]:
from dateutil import parser
frames=[]

for url in urls:
    response=requests.get(url)
    page=response.text
    soup=BeautifulSoup(page, 'lxml')
    songs=get_songs(soup)
    artists=get_artists(soup)
    dates = [parser.parse(url.split('/')[5])]*len(artists)
    df = pd.DataFrame({'artist': artists, 'track': songs, 'publish_date': dates})
    frames.append(df)

In [ ]:
df_merge = pd.concat(frames).reset_index(drop=True)
df2 = df_merge.drop_duplicates(['track'], keep='first').reset_index(drop=True)
